In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
%pylab inline
import copy
from googletrans import Translator
import pandas as pd
import numpy as np
import csv
import re
import jieba
import time
import json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import linear_model
from sklearn.externals import joblib
from nltk.corpus import stopwords as e_stopwords
from datetime import datetime, timedelta
import jieba
import sys

from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

Populating the interactive namespace from numpy and matplotlib


C:\Users\DELL\Anaconda3\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['copy', 'datetime']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


# Random Forest Regressor for Statistical Factors

In [3]:
import import_ipynb
from evaluation import precision
from runTime import runTime

importing Jupyter notebook from evaluation.ipynb
importing Jupyter notebook from runTime.ipynb


In [4]:
#Reading data from document
import pandas as pd
df_pre=pd.read_csv("E:\\DMA_PRE\\PREPROCESSED.csv")

In [21]:
df_pre.shape[0]

37263

In [17]:
train=df_pre[0:8000]
cv=df_pre[8001:10000]

In [19]:
df_stat=pd.read_csv("E:\\DMA_PRE\\train_uid_stat.csv")


In [20]:
df_stat.shape

(37263, 13)

In [62]:
df_stat.head(5)

,u_id,forward_min,forward_max,forward_median,forward_mean,comment_min,comment_max,comment_median,comment_mean,like_min,like_max,like_median,like_mean
0,000127c6126e2b0019f255ed21ac1cb7,0,1,0,0,0,0,0,0,0,0,0,0
1,0001565a5edece1669577e2ace9a6a3d,0,0,0,0,0,1,0,0,0,0,0,0
2,00033a6513b86b2705de9ffa9d37ffb6,0,0,0,0,0,0,0,0,0,1,0,0
3,0004fe2742507420eaa73e119dc83ac5,0,6,0,0,0,1,0,0,0,1,0,0
4,000c663a24a2f91f4ba156fcd4f8b9f2,0,1,0,0,0,7,0,0,0,6,0,0


In [4]:
train_all1=pd.read_csv('E:\\5th-Sem\\DMA Project\\Model Evaluation\\weibo_train1_cp.csv')
train_all2=pd.read_csv('E:\\5th-Sem\\DMA Project\\Model Evaluation\\weibo_train2_cp.csv')
frames=[train_all1,train_all2]
train_all=pd.concat(frames)

In [71]:
train_all.shape[0]

1229618

In [73]:
df_merge = pd.merge(train_all, df_stat, how='left', on=['u_id'])

In [5]:
df1=pd.read_csv("E:\\5th-Sem\\DMA Project\\Project\\weibo_train1_cpts.csv")
df2=pd.read_csv("E:\\5th-Sem\\DMA Project\\Project\\weibo_train2_cpts.csv")
frames=[df1,df2]
train_all=pd.concat(frames)

In [6]:
X=train_all[["content_media_count","content_#_count","content_length","content_emoji_count","forward_median","comment_median","like_median"]]
y=train_all[['forward_count', 'comment_count', 'like_count']]

In [7]:
from sklearn import cross_validation
## Spliting of training dataset into 70% training data and 30% testing data randomly
features_train, features_test, labels_train, labels_test = cross_validation.train_test_split(X, y, test_size=0.3, random_state=1)



C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Model 1 (Predicting all 3 values together)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

x = features_train
y = labels_train
x1 = features_test
y1 = labels_test



regr = RandomForestRegressor(max_depth=30, random_state=0,n_estimators=100)
regr.fit(x, y)
y11_predict = regr.predict(x1)
y11_predict=y11_predict.round()
y11_predict=(np.maximum(y11_predict,0.))
#print(r2_score(y1, y11_predict) ) #Random forest regressor

In [16]:
np.savetxt("E:\\weibo_predict_result.csv",y11_predict,delimiter=',',header="forward_count,comment_count,like_count",comments="")
result=pd.read_csv("E:\\weibo_predict_result.csv")


In [20]:
train_real_pred = labels_test
forward=result['forward_count'].values
comment=result['comment_count'].values
like=result['like_count'].values
train_real_pred['fp'],train_real_pred['cp'],train_real_pred['lp'] = forward,comment,like
print ("Score on the training set:{0:.2f}%".format(precision(train_real_pred.values)*100))

Score on the training set:30.04%


In [24]:
train_all.head(1).T

,0
u_id,d38e9bed5d98110dc2489d0d1cac3c2a
m_id,7d45833d9865727a88b960b0603c19f6
forward_count,0
comment_count,0
like_count,0
content,丽江旅游(sz002033)#股票##炒股##财经##理财##投资#推荐包赢股，盈利对半分成...
date,2015-02-23
time,17:41:29
content_media_count,0
content_#_count,10


# Model 2 - Constructing 3 individual models and concatinating the results

In [7]:
X=train_all[["forward_median","forward_mean","forward_min","content_media_count","content_emoji_count"]]
y=train_all['forward_count']
from sklearn import cross_validation
## Spliting of training dataset into 70% training data and 30% testing data randomly
features_train, features_test, labels_train, labels_test = cross_validation.train_test_split(X, y, test_size=0.3, random_state=1)
from sklearn.ensemble import RandomForestRegressor

x = features_train
y = labels_train
x1 = features_test
y1 = labels_test



regr = RandomForestRegressor(max_depth=30, random_state=0,n_estimators=100)
regr.fit(x, y)
y11_predict = regr.predict(x1)
#print(r2_score(y1, y11_predict) ) #Random forest regressor
np.savetxt("E:\\weibo_predict_result1.csv",y11_predict,delimiter=',',header="forward_count",comments="")
result1=pd.read_csv("E:\\weibo_predict_result1.csv")


C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [11]:
X=train_all[["comment_median","comment_mean","comment_min","content_media_count","content_emoji_count"]]
y=train_all['comment_count']
from sklearn import cross_validation
## Spliting of training dataset into 70% training data and 30% testing data randomly
features_train, features_test, labels_train, labels_test = cross_validation.train_test_split(X, y, test_size=0.3, random_state=1)
from sklearn.ensemble import RandomForestRegressor

x = features_train
y = labels_train
x1 = features_test
y2 = labels_test



regr = RandomForestRegressor(max_depth=30, random_state=0,n_estimators=100)
regr.fit(x, y)
y11_predict = regr.predict(x1)
#print(r2_score(y1, y11_predict) ) #Random forest regressor
np.savetxt("E:\\weibo_predict_result2.csv",y11_predict,delimiter=',',header="comment_count",comments="")
result2=pd.read_csv("E:\\weibo_predict_result2.csv")

In [12]:
X=train_all[["like_median","like_mean","like_min","content_media_count","content_emoji_count"]]
y=train_all['like_count']
from sklearn import cross_validation
## Spliting of training dataset into 70% training data and 30% testing data randomly
features_train, features_test, labels_train, labels_test = cross_validation.train_test_split(X, y, test_size=0.3, random_state=1)
from sklearn.ensemble import RandomForestRegressor

x = features_train
y = labels_train
x1 = features_test
y3 = labels_test



regr = RandomForestRegressor(max_depth=30, random_state=0,n_estimators=100)
regr.fit(x, y)
y11_predict = regr.predict(x1)
#print(r2_score(y1, y11_predict) ) #Random forest regressor
np.savetxt("E:\\weibo_predict_result3.csv",y11_predict,delimiter=',',header="like_count",comments="")
result3=pd.read_csv("E:\\weibo_predict_result3.csv")

In [13]:
train_real_pred = pd.concat([y1,y2,y3],axis=1)
train_real_pred['fp']=result1['forward_count']
train_real_pred['cp']=result2['comment_count']
train_real_pred['lp']=result3['like_count']
train_real_pred=train_real_pred.round()
print ("Score on the training set:{0:.4f}%".format(precision(train_real_pred.values)*100))

Score on the training set:12.1019%
